<a href="https://colab.research.google.com/github/Regina-Arthur/Python-Coding-Projects/blob/main/Pytorch_tutorial/Lenet_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [ ]:
class lenet(nn.Module):
  def__init__(self, classes = 2):
    super().__init__()
    #The convolutional layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5)

    #The pooling layer
    self.pool = nn.AvgPool2d(kernel_size = 2, stride = 2)

    #The flattening layer
    self.flatten = nn.Flatten()

    #The fully connected layer
    self.FC1 = nn.Linear(in_features = 120, out_features = 84)
    self.FC2 = nn.Linear(in_features = 84, out_features = classes)

  #The forward pass
  def forward(self,x):
    x = self.pool(F.tanh(self.conv1(x)))
    x = self.pool(F.tanh(self.conv2(x)))
    x = self.flatten(F.tanh(self.conv3(x)))
    x = F.tanh(self.FC1(x))
    x = self.FC2(x)
    return x